In [1]:
import pandas as pd
print(pd.__version__)


2.3.3


Manual spotting of unusual preprocessing is hard because workflows are heterogeneous and sparse. 
Workflow flags -
Each flag answers: “Did this preprocessing step actually happen for this sample?”

| Flag              | Meaning                                  |
| ----------------- | ---------------------------------------- |
| `did_dry`         | Sample was dried (`dryWeightNet` exists) |
| `did_ash`         | Sample was ashed (`ashWeightNet` exists) |
| `did_vaporize`    | Vaporization step performed              |
| `did_sieve`       | Sieving performed                        |
| `did_add_carrier` | Sr carrier added                         |
| `vacuumed`        | Sample was vacuumed                      |


In [3]:
import pandas as pd

# Load dataset 1
df = pd.read_csv("dataset1_280126.csv")

# Define workflow flags
flags = pd.DataFrame({
    "pseudoid": df["pseudoid"],
    "did_dry": df["dryWeightNet"].notna(),
    "did_ash": df["ashWeightNet"].notna(),
    "did_vaporize": df["totalVaporizationAmount"].notna() | df["totalVaporizationVolume"].notna(),
    "did_sieve": df["sievedDryWeight"].notna() | df["sievedOrganicWeight"].notna() | df["sievedRocksWeight"].notna(),
    "did_add_carrier": df["SrCarrierVolume"].notna(),
    "vacuumed": df["vacuumed"] == 1
})

# Summarize counts
summary = flags.drop(columns="pseudoid").sum().to_frame(name="number_of_samples")
summary["percentage"] = (summary["number_of_samples"] / len(df) * 100).round(2)

summary


,number_of_samples,percentage
did_dry,5290,23.88
did_ash,1141,5.15
did_vaporize,3332,15.04
did_sieve,214,0.97
did_add_carrier,1524,6.88
vacuumed,2072,9.35


Dataset 1 shows that preprocessing procedures are highly heterogeneous and that missing values systematically encode different laboratory workflows rather than random data quality issues. Only about one quarter of samples (≈24%) underwent drying, while more advanced steps such as ashing (≈5%), carrier addition (≈7%), and sieving (≈1%) were applied to relatively small and specific subsets of samples. Vaporization was performed for approximately 15% of samples, and vacuuming for about 9%, indicating additional but non-standard preprocessing variations. These results demonstrate that most samples follow minimal preprocessing pipelines, while a smaller number are processed using specialized or complex workflows. Consequently, preprocessing steps must be explicitly modeled (e.g., via workflow flags) before applying clustering or classification methods, as treating missing values as noise or imputing them indiscriminately would obscure meaningful structure in the data.

By defining workflow flags, we’ve learned:

- Which preprocessing steps were performed

- That missing values are intentional

- That samples follow distinct preprocessing pipelines

So now we stop treating Dataset 1 as “one table” and start treating it as multiple workflows.

In [4]:
# --- Recreate workflow flags ---
df["did_dry"] = df["dryWeightNet"].notna()
df["did_ash"] = df["ashWeightNet"].notna()
df["did_vaporize"] = df["totalVaporizationAmount"].notna() | df["totalVaporizationVolume"].notna()
df["did_sieve"] = (
    df["sievedDryWeight"].notna()
    | df["sievedOrganicWeight"].notna()
    | df["sievedRocksWeight"].notna()
)
df["did_add_carrier"] = df["SrCarrierVolume"].notna()

# --- Consistency checks ---
checks = {}

# 1. Ashing without drying
#checks["ash_without_dry"] = df["did_ash"] & (~df["did_dry"])

# 2. Gross mass smaller than tare mass
checks["gross_less_than_tare"] = (
    df["massInJarGross"].notna()
    & df["massInJarTare"].notna()
    & (df["massInJarGross"] < df["massInJarTare"])
)

# 3. Dry weight larger than net_wet
checks["dry_gt_netwet"] = (
    df["dryWeightNet"].notna()
    & df["weightNet"].notna()
    & (df["dryWeightNet"] > df["weightNet"])
)

# 4. Ash weight larger than dry weight
checks["ash_gt_dry"] = (
    df["ashWeightNet"].notna()
    & df["dryWeightNet"].notna()
    & (df["ashWeightNet"] > df["dryWeightNet"])
)

# 5. Dry weight percent > 100
checks["dry_percent_gt_100"] = (
    df["dryWeightPercent"].notna()
    & (df["dryWeightPercent"] > 100)
)

# 6. Non-positive density
checks["density_nonpositive"] = (
    df["density"].notna()
    & (df["density"] <= 0)
)

# 7. Tare greater than gross
checks["tare_gt_gross"] = (
    df["massInJarGross"].notna()
    & df["massInJarTare"].notna()
    & (df["massInJarTare"] > df["massInJarGross"])
)


# --- Combine checks ---
checks_df = pd.DataFrame(checks)
df["is_consistent"] = ~checks_df.any(axis=1)

# Count violations
violation_summary = checks_df.sum().to_frame(name="number_of_violations")
violation_summary["percentage_of_samples"] = (
    violation_summary["number_of_violations"] / len(df) * 100
).round(2)

violation_summary


,number_of_violations,percentage_of_samples
gross_less_than_tare,10,0.05
dry_gt_netwet,8,0.04
ash_gt_dry,0,0.00
dry_percent_gt_100,10,0.05
density_nonpositive,10,0.05
tare_gt_gross,10,0.05


### Within-Workflow Distribution Analysis

In [5]:
dry_df = df[df["did_dry"]]
ash_df = df[df["did_ash"]]
vap_df = df[df["did_vaporize"]]

In [6]:
dry_stats = dry_df["dryWeightNet"].describe()
dry_stats


count    5290.000000
mean      306.261905
std       408.519720
min         0.160000
25%        44.227500
50%       121.335000
75%       424.792500
max      3595.600000
Name: dryWeightNet, dtype: float64

In [7]:
ash_df = ash_df[
    ash_df["dryWeightNet"].notna() & 
    (ash_df["dryWeightNet"] > 0)
]

ash_df["ash_fraction"] = (
    ash_df["ashWeightNet"] / ash_df["dryWeightNet"]
)

ash_df["ash_fraction"].describe()


count    168.000000
mean       0.048216
std        0.028357
min        0.023080
25%        0.038176
50%        0.041554
75%        0.046180
max        0.270833
Name: ash_fraction, dtype: float64

In [8]:
Q1 = dry_df["dryWeightNet"].quantile(0.25)
Q3 = dry_df["dryWeightNet"].quantile(0.75)
IQR = Q3 - Q1

dry_outliers = dry_df[
    (dry_df["dryWeightNet"] < Q1 - 1.5 * IQR) |
    (dry_df["dryWeightNet"] > Q3 + 1.5 * IQR)
]

len(dry_outliers)


403

In [9]:
df.groupby("did_dry")["density"].describe()


,count,mean,std,min,25%,50%,75%,max
did_dry,,,,,,,,
False,14624.0,0.812776,0.566421,0.0,0.56,0.80,1.0,20.93
True,5142.0,0.954829,0.468345,0.1,0.65,0.83,1.2,5.37


In [10]:
df.groupby(["did_dry", "did_ash"])["density"].mean()


did_dry  did_ash
False    False      0.832716
         True       0.528319
True     False      0.966897
         True       0.579063
Name: density, dtype: float64

In [11]:
workflow_cols = [
    "did_dry",
    "did_ash",
    "did_vaporize",
    "did_sieve",
    "did_add_carrier",
    "vacuumed"
]

pipeline_counts = (
    df.groupby(workflow_cols)
      .size()
      .reset_index(name="count")
      .sort_values("count", ascending=False)
)

pipeline_counts.head(10)


,did_dry,did_ash,did_vaporize,did_sieve,did_add_carrier,vacuumed,count
0,False,False,False,False,False,0,12996
11,True,False,False,False,False,0,3437
5,False,False,True,False,True,0,1508
12,True,False,False,False,False,1,1468
8,False,True,True,False,False,0,952
3,False,False,True,False,False,0,850
1,False,False,False,False,False,1,522
16,True,True,False,False,False,0,167
13,True,False,False,True,False,0,143
14,True,False,False,True,False,1,71


### Finding out unique workflows

In [15]:

workflow_combinations = (
    df[workflow_cols + ["pseudoid"]]
      .drop_duplicates()
      .groupby(workflow_cols)
      .size()
      .to_frame(name="frequency")
      .reset_index()
      .sort_values("frequency", ascending=False)
)

In [16]:
def build_workflow_label(row):
    steps = []
    if row["did_dry"]:
        steps.append("DRY")
    if row["did_ash"]:
        steps.append("ASH")
    if row["did_vaporize"]:
        steps.append("VAP")
    if row["did_sieve"]:
        steps.append("SIEVE")
    if row["did_add_carrier"]:
        steps.append("CARRIER")
    if row["vacuumed"]:
        steps.append("VAC")
        
    return " → ".join(steps) if steps else "RAW"

workflow_combinations["workflow_label"] = workflow_combinations.apply(build_workflow_label, axis=1)

workflow_combinations


,did_dry,did_ash,did_vaporize,did_sieve,did_add_carrier,vacuumed,frequency,workflow_label
0,False,False,False,False,False,0,12996,RAW
11,True,False,False,False,False,0,3437,DRY
5,False,False,True,False,True,0,1508,VAP → CARRIER
12,True,False,False,False,False,1,1468,DRY → VAC
8,False,True,True,False,False,0,952,ASH → VAP
3,False,False,True,False,False,0,850,VAP
1,False,False,False,False,False,1,522,VAC
16,True,True,False,False,False,0,167,DRY → ASH
13,True,False,False,True,False,0,143,DRY → SIEVE
14,True,False,False,True,False,1,71,DRY → SIEVE → VAC


In [17]:
unique_workflows = workflow_combinations["workflow_label"].unique()

for wf in unique_workflows:
    print(wf)

RAW
DRY
VAP → CARRIER
DRY → VAC
ASH → VAP
VAP
VAC
DRY → ASH
DRY → SIEVE
DRY → SIEVE → VAC
ASH
ASH → VAP → CARRIER
CARRIER
VAP → VAC
DRY → VAP
ASH → VAP → VAC
ASH → VAC
DRY → ASH → VAP
